# Implement the anzats

In [8]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, CXGate, CZGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

In [71]:
def anzats_circ(thetas, phis, D1, D2):
    qr = QuantumRegister(4, name="q")
    cr = ClassicalRegister(4, name='c')
    qc = QuantumCircuit(qr, cr)
    for d in range(D1):
        qc.append(RXGate(phis[0]), [qr[2]])
        qc.append(RXGate(phis[1]), [qr[3]])
        
        qc.append(RZGate(phis[2]), [qr[2]])
        qc.append(RZGate(phis[3]), [qr[3]])
        
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
        
    for d in range(D2):
        qc.append(RXGate(thetas[0]), [qr[0]])
        qc.append(RXGate(thetas[1]), [qr[1]])
        qc.append(RXGate(thetas[2]), [qr[2]])
        qc.append(RXGate(thetas[3]), [qr[3]])
        
        qc.append(RZGate(thetas[4]), [qr[0]])
        qc.append(RZGate(thetas[5]), [qr[1]])
        qc.append(RZGate(thetas[6]), [qr[2]])
        qc.append(RZGate(thetas[7]), [qr[3]])
        
        qc.append(CZGate(), [qr[0], qr[1]])
        qc.append(CZGate(), [qr[1], qr[2]])
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
    
    qc.append(RXGate(thetas[0]), [qr[0]])
    qc.append(RXGate(thetas[1]), [qr[1]])
    qc.append(RXGate(thetas[2]), [qr[2]])
    qc.append(RXGate(thetas[3]), [qr[3]])

    qc.append(RZGate(thetas[4]), [qr[0]])
    qc.append(RZGate(thetas[5]), [qr[1]])
    qc.append(RZGate(thetas[6]), [qr[2]])
    qc.append(RZGate(thetas[7]), [qr[3]])
    
    #qc.measure_all()
    qc.save_statevector()
    
    return qc

# Choose k orthogonal states(computational basis)

In [72]:
import numpy as np
def get_k_basis(k, n):
    full_basis = np.identity(n)
    return full_basis[:k]


# Calculating the target function

In [103]:
from qiskit import assemble, Aer
import math
sim = Aer.get_backend('aer_simulator')

def calc_target_func1(theta, basis, D1, D2, Ham):
    target_func = 0
    for j in basis:
        qc = anzats_circ(theta[0:8], theta[8:], D1, D2)
        qc.initialize(j)
        print(qc.draw())
        qobj = assemble(qc)
        state = sim.run(qobj).result().get_statevector()
        dagger_state = np.matrix(state)
        state = dagger_state.getH()
        state = np.array(state)
        dagger_state = np.array(dagger_state)
        print("--- state vectors now: ---")
        print(state)
        print(dagger_state)
        print("--- state vectors ends. ---")
        
        product = np.matmul(Ham, state)
        product = np.matmul(dagger_state, product)

        target_func += product
    return target_func



In [105]:
theta = np.zeros(12)
basis = get_k_basis(4,16)
target_func = calc_target_func1(theta, basis, 1, 1, np.identity(16))
print("target func:")
print(target_func)

                           ░ ┌───────┐┌───────┐          ░ ┌───────┐┌───────┐»
q_0: ──────────────────────░─┤ Rx(0) ├┤ Rz(0) ├─■────────░─┤ Rx(0) ├┤ Rz(0) ├»
                           ░ ├───────┤├───────┤ │        ░ ├───────┤├───────┤»
q_1: ──────────────────────░─┤ Rx(0) ├┤ Rz(0) ├─■──■─────░─┤ Rx(0) ├┤ Rz(0) ├»
     ┌───────┐┌───────┐    ░ ├───────┤├───────┤    │     ░ ├───────┤├───────┤»
q_2: ┤ Rx(0) ├┤ Rz(0) ├─■──░─┤ Rx(0) ├┤ Rz(0) ├────■──■──░─┤ Rx(0) ├┤ Rz(0) ├»
     ├───────┤├───────┤ │  ░ ├───────┤├───────┤       │  ░ ├───────┤├───────┤»
q_3: ┤ Rx(0) ├┤ Rz(0) ├─■──░─┤ Rx(0) ├┤ Rz(0) ├───────■──░─┤ Rx(0) ├┤ Rz(0) ├»
     └───────┘└───────┘    ░ └───────┘└───────┘          ░ └───────┘└───────┘»
c: 4/════════════════════════════════════════════════════════════════════════»
                                                                             »
«      ░ ┌──────────────────────────────────────────────┐
«q_0: ─░─┤0                                             ├
«      ░ │     

# Sending the target function to the BFGS optimazer 